# Model Code


In [4]:
import sqlite3

try:
    conn = sqlite3.connect('spare_parts3.db')
    conn.close()  # Close any active connection
except Exception as e:
    print("Error closing the database:", e)


In [5]:
import sqlite3
import datetime
import math
from tkinter import Tk, Label, Entry, Button, Toplevel, messagebox, Text
import os
if os.path.exists("spare_parts3.db"):
    os.remove("spare_parts3.db")


# Create Database Table if not exists
def create_table():
    conn = sqlite3.connect('spare_parts3.db')
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS SpareParts (
            part_id TEXT PRIMARY KEY,
            part_name TEXT,
            installation_date TEXT,
            age_threshold INTEGER,
            demand_rate REAL,
            holding_cost REAL,
            ordering_cost REAL,
            num_installed INTEGER,
            next_reorder_date TEXT
        )
    """)
    conn.commit()
    conn.close()

# SparePart Class for calculations
class SparePart:
    def __init__(self, part_id, part_name, installation_date, age_threshold, demand_rate, holding_cost, ordering_cost, num_installed):
        self.part_id = part_id
        self.part_name = part_name
        self.installation_date = datetime.datetime.strptime(installation_date, '%Y-%m-%d')
        self.age_threshold = age_threshold
        self.demand_rate = demand_rate
        self.holding_cost = holding_cost
        self.ordering_cost = ordering_cost
        self.num_installed = num_installed

    def calculate_eoq(self):
        return round(math.sqrt((2 * self.demand_rate * self.ordering_cost) / self.holding_cost), 2)

    def calculate_reorder_point(self):
        #Calculate the reorder point based on age threshold and demand rate.
        daily_demand = self.demand_rate / 365
        #Lead time is used to determine how early you should reorder a part before it exceeds its age threshold.
        lead_time = self.age_threshold * 0.8 # Assuming reorder at 80% of the age threshold
        return round(daily_demand * lead_time, 2)

    def next_reorder_date(self):
        return (self.installation_date + datetime.timedelta(days=self.age_threshold * 0.8)).strftime('%Y-%m-%d')

    def save_to_db(self):
        conn = sqlite3.connect('spare_parts3.db')
        cursor = conn.cursor()
        cursor.execute("""
            INSERT OR REPLACE INTO SpareParts 
            (part_id, part_name, installation_date, age_threshold, demand_rate, holding_cost, ordering_cost, num_installed, next_reorder_date)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            self.part_id, self.part_name, self.installation_date.strftime('%Y-%m-%d'),
            self.age_threshold, self.demand_rate, self.holding_cost,
            self.ordering_cost, self.num_installed, self.next_reorder_date()
        ))
        conn.commit()
        conn.close()

# Function to add new spare parts
def add_part_window():
    def save_part():
        part = SparePart(
            entry_id.get(), entry_name.get(), entry_date.get(), int(entry_threshold.get()),
            float(entry_demand.get()), float(entry_holding.get()), float(entry_ordering.get()), int(entry_stock.get())
        )
        part.save_to_db()
        messagebox.showinfo("Success", "Spare part added successfully!")
        window.destroy()

    window = Toplevel()
    window.title("Add New Spare Part")
    window.geometry("400x400")
    
    for text, var in zip(["Part ID", "Part Name", "Installation Date (YYYY-MM-DD)", "Age Threshold", "Annual Demand", "Holding Cost", "Ordering Cost", "Stock"],
                          ["entry_id", "entry_name", "entry_date", "entry_threshold", "entry_demand", "entry_holding", "entry_ordering", "entry_stock"]):
        Label(window, text=f"{text}:").pack(pady=5)
        globals()[var] = Entry(window)
        globals()[var].pack(pady=5)
    
    Button(window, text="Save Part", command=save_part).pack(pady=20)

# Function to view inventory
def view_inventory():
    window = Toplevel()
    window.title("Inventory")
    window.geometry("500x400")
    
    text = Text(window, wrap="word")
    text.pack(expand=True, fill="both")
    
    conn = sqlite3.connect('spare_parts3.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM SpareParts")
    parts = cursor.fetchall()
    conn.close()
    
    for part in parts:
        text.insert("end", f"ID: {part[0]}, Name: {part[1]}, Stock: {part[7]}, Next Reorder: {part[8]}\n")
    text.config(state="disabled")

# Function to check reorder points
def check_reorder_dates():
    conn = sqlite3.connect('spare_parts3.db')
    cursor = conn.cursor()
    today = datetime.datetime.now().strftime('%Y-%m-%d')
    cursor.execute("SELECT * FROM SpareParts WHERE next_reorder_date <= ?", (today,))
    parts = cursor.fetchall()
    conn.close()
    
    message = "Parts needing reorder:\n" + "\n".join([f"{p[1]} (ID: {p[0]}) - Next Reorder: {p[8]}" for p in parts]) if parts else "No parts need reordering today."
    messagebox.showinfo("Reorder Notification", message)

# Function to handle urgent repair
def urgent_repair_window():
    def process_repair():
        part_id, quantity_used = entry_id.get(), entry_quantity.get()
        if not quantity_used.isdigit():
            messagebox.showerror("Error", "Quantity Used must be a valid number!")
            return
        quantity_used = int(quantity_used)
        conn = sqlite3.connect('spare_parts3.db')
        cursor = conn.cursor()
        cursor.execute("SELECT num_installed FROM SpareParts WHERE part_id = ?", (part_id,))
        result = cursor.fetchone()
        if result and quantity_used <= result[0]:
            cursor.execute("UPDATE SpareParts SET num_installed = ? WHERE part_id = ?", (result[0] - quantity_used, part_id))
            conn.commit()
            messagebox.showinfo("Success", f"Repair recorded! Remaining stock: {result[0] - quantity_used}")
        else:
            messagebox.showerror("Error", "Invalid part ID or insufficient stock!")
        conn.close()
    
    window = Toplevel()
    window.title("Urgent Repair")
    window.geometry("400x300")
    Label(window, text="Part ID:").pack(pady=5)
    entry_id = Entry(window)
    entry_id.pack(pady=5)
    Label(window, text="Quantity Used:").pack(pady=5)
    entry_quantity = Entry(window)
    entry_quantity.pack(pady=5)
    Button(window, text="Record Repair", command=process_repair).pack(pady=20)

# Main GUI
def main():
    root = Tk()
    root.title("Inventory Management System")
    root.geometry("500x400")
    
    Label(root, text="Spare Parts Inventory System", font=("Helvetica", 16)).pack(pady=20)
    for text, command in zip(["Add New Spare Part", "View Inventory", "Check Reorder Notifications", "Urgent Repair"],
                              [add_part_window, view_inventory, check_reorder_dates, urgent_repair_window]):
        Button(root, text=text, command=command, width=30).pack(pady=10)
    
    root.mainloop()

if __name__ == "__main__":
    create_table()
    main()



# View Database

In [2]:
import sqlite3

def view_database():
    """Display all entries in the SpareParts database, including stock, EOQ, and reorder point."""
    try:
        # Connect to the database
        conn = sqlite3.connect('spare_parts3.db')
        cursor = conn.cursor()

        # Check if the SpareParts table exists
        cursor.execute("""
            SELECT name FROM sqlite_master WHERE type='table' AND name='SpareParts';
        """)
        table_exists = cursor.fetchone()

        if not table_exists:
            print("No data found. The SpareParts table does not exist.")
            return

        # Query all entries in the SpareParts table
        cursor.execute("SELECT * FROM SpareParts")
        rows = cursor.fetchall()

        if not rows:
            print("No entries found in the database.")
        else:
            print("\nSpare Parts Inventory:")
            print("-" * 130)
            print(f"{'Part ID':<15}{'Part Name':<20}{'Install Date':<15}{'Age Threshold':<15}{'Stock':<10}{'EOQ':<10}{'Reorder Point':<15}{'Next Reorder Date'}")
            print("-" * 130)
            for row in rows:
                part_id, part_name, install_date, age_threshold, demand_rate, holding_cost, ordering_cost, stock, next_reorder = row
                
                # Calculate EOQ
                eoq = round(((2 * demand_rate * ordering_cost) / holding_cost) ** 0.5, 2) if holding_cost > 0 else 0
                
                # Calculate Reorder Point
                daily_demand = demand_rate / 365
                lead_time = age_threshold * 0.8
                reorder_point = round(daily_demand * lead_time, 2)
                
                print(f"{part_id:<15}{part_name:<20}{install_date:<15}{age_threshold:<15}{stock:<10}{eoq:<10}{reorder_point:<15}{next_reorder}")
    except Exception as e:
        print(f"Error accessing the database: {e}")
    finally:
        conn.close()

if __name__ == "__main__":
    view_database()



Spare Parts Inventory:
----------------------------------------------------------------------------------------------------------------------------------
Part ID        Part Name           Install Date   Age Threshold  Stock     EOQ       Reorder Point  Next Reorder Date
----------------------------------------------------------------------------------------------------------------------------------
P0001          Bolt                2024-09-01     190            850       447.21    416.44         2025-01-31
P0002          Wings               2025-01-01     365            50        70.71     40.0           2025-10-20
P0003          Screws              2025-02-15     183            980       223.61    401.1          2025-07-11
